<a href="https://colab.research.google.com/github/monoramasn/speech-recognition/blob/main/edacc_whisper_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git@main
!pip install -U accelerate
!pip install evaluate
!pip install librosa
!pip install jiwer
!pip install --upgrade transformers bitsandbytes datasets torch torchvision torchaudio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 60.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully 

In [ ]:
import os
import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim
import argparse
import evaluate
import re
from scipy import signal
import numpy as np
import pandas as pd
from dataclasses import dataclass
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from typing import Any, Dict, List, Union
from sklearn.model_selection import train_test_split
from datasets import load_dataset, DatasetDict, Audio, load_from_disk, concatenate_datasets, load_metric
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperTokenizer, WhisperFeatureExtractor
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
from datasets import load_dataset

# Load datasets
edacc_dev = load_dataset("edinburghcstr/edacc", split="validation")
edacc_test = load_dataset("edinburghcstr/edacc", split="test")

# Extract unique languages (l1) from each split
val_languages = edacc_dev.unique("l1")
test_languages = edacc_test.unique("l1")

# Print the number of unique languages and the languages themselves
print("Number of unique languages in validation split:", len(val_languages))
print("Languages in validation split:", val_languages)

print("Number of unique languages in test split:", len(test_languages))
print("Languages in test split:", test_languages)


Number of unique languages in validation split: 29
Languages in validation split: ['Southern British English', 'Mainstream US English', 'Polish', 'Indian English', 'Maltese', 'Italian', 'German', 'Dutch', 'Japanese', 'Mandarin', 'Icelandic', 'French', 'Bulgarian', 'Catalan', 'Russian', 'Korean', 'Spanish', 'Romanian', 'Montenegrin', 'English', 'Bahasa', 'Arabic', 'South African English', 'Macedonian', 'Scottish English', 'Jamaican English', 'Irish English', 'Portoguese', 'Vietnamese']
Number of unique languages in test split: 26
Languages in test split: ['Scottish English', 'Sinhalese', 'Lithuanian', 'Bulgarian', 'Mandarin', 'Mainstream US English', 'Catalan', 'Spanish', 'Romanian', 'Indian English', 'Nigerian English', 'French', 'Spanish (Mexican)', 'Urdu', 'Kenyan English', 'Ghanain English', 'Hindi', 'Filipino', 'Tagalog', 'Southern British English', 'Irish English', 'Jamaican English', 'Italian', 'Hebrew', 'Vietnamese', 'Indonesian']


In [ ]:
# Load datasets
edacc_dev = load_dataset("edinburghcstr/edacc", split="validation")
edacc_test = load_dataset("edinburghcstr/edacc", split="test")

# Create directories for each language in the test, train, and validation sets
base_dir = "organized_data"
os.makedirs(base_dir, exist_ok=True)

languages = edacc_test.unique("l1")
for split in ['test', 'train', 'validation']:
    for lang in languages:
        os.makedirs(os.path.join(base_dir, split, lang), exist_ok=True)

# Function to save audio data and corresponding text
def save_audio(example, folder):
    lang_folder = os.path.join(folder, example["l1"])
    os.makedirs(lang_folder, exist_ok=True)
    audio_filename = example['audio']['path'].split('/')[-1]
    audio_path = os.path.join(lang_folder, audio_filename)
    # Save audio file
    torchaudio.save(audio_path, torch.tensor(example['audio']['array']).unsqueeze(0), example['audio']['sampling_rate'])
    # Save corresponding text file
    text_path = os.path.join(lang_folder, f"{audio_filename}.txt")
    with open(text_path, "w") as f:
        f.write(example["text"])
    print(f"Saved audio: {audio_path}")
    print(f"Saved text: {text_path}")

# Save test audio and text to respective language directories
print("Saving test data...")
for example in edacc_test:
    save_audio(example, os.path.join(base_dir, 'test'))

# Split the validation dataset into train and validation sets
X = range(len(edacc_dev))
X_train, X_val = train_test_split(X, test_size=0.2, random_state=42)

train_dataset = edacc_dev.select(X_train)
val_dataset = edacc_dev.select(X_val)

# Save train audio and text to respective language directories
print("Saving train data...")
for example in train_dataset:
    save_audio(example, os.path.join(base_dir, 'train'))

# Save validation audio and text to respective language directories
print("Saving validation data...")
for example in val_dataset:
    save_audio(example, os.path.join(base_dir, 'validation'))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating validation split:   0%|          | 0/9848 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9289 [00:00<?, ? examples/s]

Streaming output truncated to the last 5000 lines.
Saved text: organized_data/train/Bulgarian/EDACC-C22-203.wav.txt
Saved audio: organized_data/train/Italian/EDACC-C01-146.wav
Saved text: organized_data/train/Italian/EDACC-C01-146.wav.txt
Saved audio: organized_data/train/Scottish English/EDACC-C46_P1-29.wav
Saved text: organized_data/train/Scottish English/EDACC-C46_P1-29.wav.txt
Saved audio: organized_data/train/Arabic/EDACC-C40_P3-40.wav
Saved text: organized_data/train/Arabic/EDACC-C40_P3-40.wav.txt
Saved audio: organized_data/train/Romanian/EDACC-C30-616.wav
Saved text: organized_data/train/Romanian/EDACC-C30-616.wav.txt
Saved audio: organized_data/train/Arabic/EDACC-C40_P3-110.wav
Saved text: organized_data/train/Arabic/EDACC-C40_P3-110.wav.txt
Saved audio: organized_data/train/Irish English/EDACC-C50-255.wav
Saved text: organized_data/train/Irish English/EDACC-C50-255.wav.txt
Saved audio: organized_data/train/Romanian/EDACC-C30-100.wav
Saved text: organized_data/train/Romanian/E

In [ ]:
from collections import defaultdict

# Function to calculate word lengths in a text
def calculate_word_lengths(text):
    words = text.split()
    word_lengths = [len(word) for word in words]
    return word_lengths

# Dictionary to store total word lengths and word counts for each language
word_length_data = defaultdict(lambda: {"total_length": 0, "word_count": 0})

# Function to process each example and update word length data
def process_example(example):
    language = example["l1"]
    word_lengths = calculate_word_lengths(example["text"])

    word_length_data[language]["total_length"] += sum(word_lengths)
    word_length_data[language]["word_count"] += len(word_lengths)

# Process the test dataset
print("Processing test data for word length calculation...")
for example in edacc_test:
    process_example(example)

# Calculate average word lengths for each language
average_word_lengths = {}
for language, data in word_length_data.items():
    if data["word_count"] > 0:
        average_word_lengths[language] = data["total_length"] / data["word_count"]
    else:
        average_word_lengths[language] = float('nan')  # Handle cases where no words were found

# Display the average word lengths for each language
for language, avg_length in average_word_lengths.items():
    print(f"Average word length for {language}: {avg_length:.2f}")


Processing test data for word length calculation...
Average word length for Scottish English: 4.02
Average word length for Sinhalese: 3.81
Average word length for Lithuanian: 3.91
Average word length for Bulgarian: 3.95
Average word length for Mandarin: 4.08
Average word length for Mainstream US English: 3.90
Average word length for Catalan: 3.95
Average word length for Spanish: 3.91
Average word length for Romanian: 3.97
Average word length for Indian English: 3.91
Average word length for Nigerian English: 3.92
Average word length for French: 3.92
Average word length for Spanish (Mexican): 3.52
Average word length for Urdu: 3.81
Average word length for Kenyan English: 4.12
Average word length for Ghanain English: 3.74
Average word length for Hindi: 3.89
Average word length for Filipino: 4.02
Average word length for Tagalog: 3.93
Average word length for Southern British English: 3.91
Average word length for Irish English: 3.97
Average word length for Jamaican English: 3.99
Average word

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Load multiple Whisper models and processors
model_names = ["openai/whisper-tiny", "openai/whisper-base", "openai/whisper-small", "openai/whisper-medium", "openai/whisper-large"]
models = {}
processors = {}
for model_name in model_names:
    processors[model_name] = WhisperProcessor.from_pretrained(model_name)
    models[model_name] = WhisperForConditionalGeneration.from_pretrained(model_name).to('cuda' if torch.cuda.is_available() else 'cpu')

# Function to downsample audio samples to 16 kHz
def downsample_audio(audio_array, source_sr, target_sr=16000):
    waveform = torch.tensor(audio_array, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
    resampled_waveform = torchaudio.transforms.Resample(source_sr, target_sr)(waveform)
    resampled_audio = resampled_waveform.squeeze(0).numpy()  # Remove channel dimension
    return resampled_audio

# Function to normalize text
def normalize_text(text):
    text = text.lower()
    text = text.strip()
    text = text.replace("\n", " ")
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove non-alphanumeric characters
    return text

# Function to evaluate WER for each language folder
def evaluate_language_wer(folder_path, processor, model, batch_size=16):
    wer_metric = load_metric("wer", trust_remote_code=True)
    audio_files = [f for f in os.listdir(folder_path) if f.endswith(".wav")]
    predictions = []
    references = []

    for i in range(0, len(audio_files), batch_size):
        batch_files = audio_files[i:i+batch_size]
        input_features = []
        for file in batch_files:
            audio_path = os.path.join(folder_path, file)
            text_path = audio_path + ".txt"
            with open(text_path, "r") as f:
                references.append(normalize_text(f.read().strip()))
            audio, sampling_rate = torchaudio.load(audio_path)
            audio = audio.squeeze().numpy()
            if sampling_rate != 16000:
                audio = downsample_audio(audio, sampling_rate)
            features = processor.feature_extractor(audio, sampling_rate=16000, return_tensors="pt").input_features
            input_features.append(features)

        input_features = torch.cat(input_features).to(model.device)
        with torch.no_grad():
            generated_ids = model.generate(input_features)
        transcriptions = processor.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        predictions.extend([normalize_text(transcription) for transcription in transcriptions])

    wer = wer_metric.compute(predictions=predictions, references=references)
    return wer

# Directory where test data is organized
base_dir = "organized_data"

# Initialize a DataFrame to store WER results
wer_results = pd.DataFrame(columns=["Language"] + model_names)

# Evaluate and print WER for each language and model
for lang in languages:
    folder_path = os.path.join(base_dir, 'test', lang)
    row = {"Language": lang}
    for model_name in model_names:
        processor = processors[model_name]
        model = models[model_name]
        wer = evaluate_language_wer(folder_path, processor, model)
        row[model_name] = wer
        print(f"Language: {lang}, Model: {model_name}, WER: {wer:.4f}")
    wer_results = pd.concat([wer_results, pd.DataFrame([row])], ignore_index=True)

# Display WER results as a table
print("\nWER Results:")
print(wer_results.to_string(index=False))

# Save the WER results to a CSV file
wer_results.to_csv("wer_results.csv", index=False)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

<ipython-input-5-ca8bdf0fbe1e>:27: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer", trust_remote_code=True)


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Language: Scottish English, Model: openai/whisper-tiny, WER: 0.4339
Language: Scottish English, Model: openai/whisper-base, WER: 0.3867
Language: Scottish English, Model: openai/whisper-small, WER: 0.3456
Language: Scottish English, Model: openai/whisper-medium, WER: 0.3121
Language: Scottish English, Model: openai/whisper-large, WER: 0.3054


<ipython-input-5-ca8bdf0fbe1e>:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  wer_results = pd.concat([wer_results, pd.DataFrame([row])], ignore_index=True)


Language: Sinhalese, Model: openai/whisper-tiny, WER: 0.6822
Language: Sinhalese, Model: openai/whisper-base, WER: 0.4551
Language: Sinhalese, Model: openai/whisper-small, WER: 0.3977
Language: Sinhalese, Model: openai/whisper-medium, WER: 0.6150
Language: Sinhalese, Model: openai/whisper-large, WER: 0.3793
Language: Lithuanian, Model: openai/whisper-tiny, WER: 0.3663
Language: Lithuanian, Model: openai/whisper-base, WER: 0.3770
Language: Lithuanian, Model: openai/whisper-small, WER: 0.3639
Language: Lithuanian, Model: openai/whisper-medium, WER: 0.2698
Language: Lithuanian, Model: openai/whisper-large, WER: 0.3007
Language: Bulgarian, Model: openai/whisper-tiny, WER: 0.7857
Language: Bulgarian, Model: openai/whisper-base, WER: 0.6805
Language: Bulgarian, Model: openai/whisper-small, WER: 0.6436
Language: Bulgarian, Model: openai/whisper-medium, WER: 0.6293
Language: Bulgarian, Model: openai/whisper-large, WER: 0.6233
Language: Mandarin, Model: openai/whisper-tiny, WER: 0.3925
Language

In [ ]:
import os
import torchaudio
import torch
from sklearn.model_selection import train_test_split

# Define base directories
base_dir = "/content/drive/MyDrive/EDACC/edacc_v1.0/organized_files"  # Update with your path
dev_dir = os.path.join(base_dir, "dev")
test_dir = os.path.join(base_dir, "test")

# Function to load audio data and corresponding text
def load_audio_and_text(audio_path, text_path):
    try:
        # Load the audio file
        audio_data, sampling_rate = torchaudio.load(audio_path)
        # Load the corresponding text file
        with open(text_path, "r") as f:
            text_content = f.read()
        return audio_data, sampling_rate, text_content
    except Exception as e:
        print(f"Error loading audio or text: {e}")
        return None, None, None

# Iterate over each directory (English variety) in the dev and test folders
for split in ['dev', 'test']:
    split_dir = os.path.join(base_dir, split)
    for lang in os.listdir(split_dir):
        lang_folder = os.path.join(split_dir, lang)
        if os.path.isdir(lang_folder):
            audio_files = [f for f in os.listdir(lang_folder) if f.endswith('.wav')]
            for audio_file in audio_files:
                audio_path = os.path.join(lang_folder, audio_file)
                text_file = audio_file.replace('.wav', '.txt')
                text_path = os.path.join(lang_folder, text_file)

                # Load the audio and text data
                audio_data, sampling_rate, text_content = load_audio_and_text(audio_path, text_path)

                # You can now process the loaded audio and text as needed
                print(f"Loaded audio for {audio_file} with text: {text_content[:50]}...")

# Split the dev data into train and validation sets for further processing
dev_files = []
for lang in os.listdir(dev_dir):
    lang_folder = os.path.join(dev_dir, lang)
    if os.path.isdir(lang_folder):
        audio_files = [os.path.join(lang_folder, f) for f in os.listdir(lang_folder) if f.endswith('.wav')]
        dev_files.extend([(audio_path, audio_path.replace('.wav', '.txt'), lang_folder) for audio_path in audio_files])

X = range(len(dev_files))
X_train, X_val = train_test_split(X, test_size=0.2, random_state=42)

# Process train and validation data
print("Processing train data...")
for i in X_train:
    audio_path, text_path, lang_folder = dev_files[i]
    lang_name = os.path.basename(lang_folder)
    audio_data, sampling_rate, text_content = load_audio_and_text(audio_path, text_path)
    print(f"Processing train audio for {audio_path} with text: {text_content[:5]}...")

print("Processing validation data...")
for i in X_val:
    audio_path, text_path, lang_folder = dev_files[i]
    lang_name = os.path.basename(lang_folder)
    audio_data, sampling_rate, text_content = load_audio_and_text(audio_path, text_path)
    print(f"Processing validation audio for {audio_path} with text: {text_content[:5]}...")

print("Processing complete.")


Streaming output truncated to the last 5000 lines.
Processing train audio for /content/drive/MyDrive/EDACC/edacc_v1.0/organized_files/dev/Korean English/EDACC-C03_187.19_208.71.wav with text: <fema...
Processing train audio for /content/drive/MyDrive/EDACC/edacc_v1.0/organized_files/dev/Mainstream US English/EDACC-C14_850.56_852.22.wav with text: <fema...
Processing train audio for /content/drive/MyDrive/EDACC/edacc_v1.0/organized_files/dev/Polish English/EDACC-C24_285.94_288.5.wav with text: <fema...
Processing train audio for /content/drive/MyDrive/EDACC/edacc_v1.0/organized_files/dev/Indonesian English/EDACC-C39_297.46_300.32.wav with text: <fema...
Processing train audio for /content/drive/MyDrive/EDACC/edacc_v1.0/organized_files/dev/Chinese English/EDACC-C06_206.04_208.51.wav with text: <male...
Processing train audio for /content/drive/MyDrive/EDACC/edacc_v1.0/organized_files/dev/Polish English/EDACC-C07_926.25_932.56.wav with text: <male...
Processing train audio for /content/dr

In [ ]:
torch.cuda.empty_cache()